<a href="https://colab.research.google.com/github/Kamalesh-Swargam/-dockingbay/blob/main/comfyui_colab_Flux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the [ComfyUI](https://github.com/comfyanonymous/ComfyUI) repo and install the requirements.

In [ ]:
!git clone -q https://github.com/comfyanonymous/ComfyUI
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu130
%pip install -q -r ./ComfyUI/requirements.txt

In [ ]:
# Some Helpful Custom Nodes
!git clone -q https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/comfyui-manager

!git clone -q https://github.com/ClownsharkBatwing/RES4LYF.git /content/ComfyUI/custom_nodes/RES4LYF
!git clone -q https://github.com/rgthree/rgthree-comfy.git /content/ComfyUI/custom_nodes/rgthree-comfy
!git clone -q https://github.com/yolain/ComfyUI-Easy-Use.git /content/ComfyUI/custom_nodes/ComfyUI-Easy-Use

Download the vae and dual text encoder needed for FLUX - (Compulsory)

In [ ]:
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors -P /content/ComfyUI/models/text_encoders/
!wget -q --show-progress https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp8_e4m3fn.safetensors -P /content/ComfyUI/models/text_encoders/
!wget -q --show-progress https://huggingface.co/cocktailpeanut/xulf-dev/resolve/main/ae.safetensors -P /content/ComfyUI/models/vae/

# 8-step distilled lora for fast inference on dev models
# !wget -q --show-progress https://huggingface.co/alimama-creative/FLUX.1-Turbo-Alpha/resolve/main/diffusion_pytorch_model.safetensors -P /content/ComfyUI/models/loras


The **FP8 version** runs smoothly on the free tier (without LoRA), but if you want to use it with a LoRA, it requires more resources than the free Colab tier provides.  
In that case, you can opt for the [gguf](https://huggingface.co/collections/QuantStack/flux-ggufs-68264cfc663d50c418940b30) quantized versions or the [Nunchaku](https://huggingface.co/nunchaku-tech/models) versions instead.

>**Note** : You need only one , either a gguf quantized model (under 10GB) or the Nunchanku model

---


In [ ]:
# %pip install --upgrade gguf
# !rm -rf ./custom_nodes/ComfyUI-GGUF
# !git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF

In [ ]:
# 1. Force install the GGUF Custom Node (Bypassing Manager)
!git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF

# 2. Move your model to the correct "Unet" folder (Where GGUF nodes look)
# (Your screenshot showed it was in diffusion_models, we move it to be safe)
!mv /content/ComfyUI/models/diffusion_models/flux1-dev-Q4_0.gguf /content/ComfyUI/models/unet/flux1-dev-Q4_0.gguf

### Run ComfyUI with ngrok

In [ ]:
!pip install -q pyngrok

from pyngrok import ngrok
import subprocess, socket, time
from google.colab import userdata

NGROK_TOKEN = userdata.get("NGROK_TOKEN")

# Set ngrok token
!ngrok config add-authtoken $NGROK_TOKEN

# Start ComfyUI in background
subprocess.Popen(["python", "/content/ComfyUI/main.py", "--dont-print-server"])

# Wait until port is open
port = 8188
while True:
    try:
        sock = socket.create_connection(("127.0.0.1", port), timeout=2)
        sock.close()
        print("✅ ComfyUI server is running on port", port)
        break
    except OSError:
        print("⏳ Waiting for ComfyUI to start...")
        time.sleep(2)

# Start ngrok tunnel
public_url = ngrok.connect(8188, bind_tls=True)
print("🌐 Public URL:", public_url)
